# Предсказание уровня заработной платы  

https://archive.ics.uci.edu/ml/datasets/adult

В данном ДЗ вам нужно будет предсказывать уровень заработной платы респондента: больше 50к долларов в год или меньше.

**Описание признаков**:

1. Категорийные признаки

`workclass`: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked. Individual work category

`education`: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. Individual's highest education degree

`marital-status`: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse. Individual marital status

`occupation`: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces. Individual's occupation

`relationship`: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried. Individual's relation in a family

`race`: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black. Race of Individual

`sex`: Female, Male.

`native-country`: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands. Individual's native country

2. Количественные признаки

`age`: continuous. Age of an individual

`fnlwgt`: final weight, continuous. The weights on the CPS files are controlled to independent estimates of the civilian noninstitutional population of the US. These are prepared monthly for us by Population Division here at the Census Bureau.

`capital-gain`: continuous.

`capital-loss`: continuous.

`hours-per-week`: continuous. Individual's working hour per week

In [7]:
import pandas as pd
import numpy as np
import os
import warnings 
import re # модуль для регулярных выражений
warnings.filterwarnings("ignore")
os.chdir("/Users/iakubovskii/Machine_Learning/RANEPA/Fintech_2020/Машинное обучение/Данные/ДЗ1/")
adult = pd.read_csv("adult.csv", dtype = {"fnlwgt": int,
                                             "Education_Num": int,
                                             "Capital_Gain":int,
                                             "Capital_Loss":int,
                                             "Hours_per_week":int,
                                             "Age":int})

In [177]:
# Список категориальных признаков
cat_cols = ['Workclass', 'Education', 'Marital_Status', 'Occupation',
           'Relationship', 'Race', 'Sex', 'Country']

# Список количественных признаков
quant_cols = list(set(adult.columns) - set(cat_cols))
quant_cols.remove("Target")
# Удалим лишние пробелы перед значениями внутри категориальных столбцов
for cat_col in cat_cols:
    adult[cat_col] = adult[cat_col].str.strip(" ")


# Преобразуем категориальные признаки при помощи дамми-кодирования
new_df = pd.concat([adult.drop(cat_cols, axis=1),
                  pd.get_dummies(adult[cat_cols], drop_first=True)],
                 axis=1)
# Удалим строки с пропусками
new_df = new_df.dropna()
# Исправим неточности в целевой переменной
print(new_df['Target'].value_counts())
new_df['Target'] = new_df['Target'].str.strip(" ")
new_df['Target'] = new_df['Target'].map({">50K.":">50K", 
                                       "<=50K.":"<=50K"}).fillna(new_df['Target'])
print(new_df['Target'].value_counts())

 <=50K     22654
 <=50K.    11360
 >50K       7508
 >50K.      3700
Name: Target, dtype: int64
<=50K    34014
>50K     11208
Name: Target, dtype: int64


##############################################################################################################
# Задания

ВНИМАНИЕ!!!
ВЕЗДЕ, где есть параметр **random_state**, устанавливайте его равным **своему номеру**, иначе у нас могут не совпасть результаты и будет плохо (кроме KFold кросс-валидации у нас будет параметр Shufle=False, а значит и random_state не нужен). 

Результаты округляем до 5 знака после запятой. Например, ROC AUC = 0.56156

Задание *найти оптимальный гиперпараметр* (если не оговорено иное) подразумевает 5  фолдовую стратифицированную кросс-валидацию с random_state, равным **вашему номеру**.

По умолчанию мы используем ВСЕ ПРИЗНАКИ из датасетов.

In [371]:
students_random_state = {
 'Базуева Мария Дмитриевна': 993,
 'Бориско Данила Ильич': 1956,
 'Братков Герман Сергеевич': 210,
 'Орлан Суван-оол': 211,
 'Валл Федор Викторович': 188,
 'Егорова Анна Сергеевна': 25,
 'Едовина Алина Игоревна': 35,
 'Загарнюк Елизавета Максимовна': 979,
 'Захаров Алексей Сергееивч': 587,
 'Калёнов Алексей Аркадьевич': 1334,
 'Карасева Алина Александровна': 1265,
 'Каширин Егор Михайлович': 1672,
 'Косинов Андрей Вячеславович': 940,
 'Красиков Евгений Владимирович': 601,
 'Кузьмин Никита Кириллович': 452,
 'Монгуш Тенгиз Анатольевич': 668,
 'Мурадян Акоп Араратович': 1155,
 'Наумова Анастасия Юрьевна': 1020,
 'Панчук Александр Сергеевич': 1125,
 'Пашинина Татьяна Викторовна': 268,
 'Пустоваров Артем Андреевич': 187,
 'Роговая Тамара Олеговна': 1472,
 'Селезнев Дмитрий Владимирович': 734,
 'Сидорякин Виталий Дмитриевич': 1554,
 'Филиппов Антон Павлович': 126,
 'Фрольцов Григорий Максимович': 1723,
 'Хамитов Давид Альбертович': 1944,
 'Хомушку Ганна Алексеевна': 582,
 'Царева Мария Сергеевна': 1336}


In [382]:
# Введите свой номер random state
your_number_random_state = 25

# Для расчета доли, по которой будем делить датасеты
max_ = max(list(map(lambda x: np.log(x), students_random_state.values())))
frac_student = np.log(your_number_random_state) / max_

# Уникальный датасет для каждого студента
df_model = new_df.sample(frac = frac_student, random_state=your_number_random_state)


# Определим целевую переменную 
X, y = df_model.drop("Target", axis=1), df_model['Target'].map({">50K":1, "<=50K":0})

In [300]:
# Разобьем на тренировочную и тестовую (уже уменьшенный датасет для каждого студента)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                   random_state=your_number_random_state)

# Стандартизируем количественные признаки

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[quant_cols] = sc.fit_transform(X_train[quant_cols])
X_test[quant_cols] = sc.transform(X_test[quant_cols])

1. При помощи метода ближайших соседей сделайте прогноз для числа соседей *со всеми объясняющими признаками*, равным 11. Чему равен `Recall` на тестовой выборке?

2. Среди следующих значений коэффициентов регуляризации (без кросс-валидации) выберите тот, для которого `ROC AUC` score на тестовой выборке для логистической регрессии *со всеми факторами* будет минимальным. В ответе укажите значение минимального ROC AUC

$$ C = [0.01, 0.05, 0.1, 0.3, 0.5, 0.75, 0.9, 1] $$

3. Чему равен `Precision` для дерева решений с параметрами по умолчанию (реализация sklearn, random_state соответствует вашему номеру)?

4. Удалите признаки, для которых коэффициенты в LASSO регрессии равны нулю (на тренировочной выборке) и сделайте прогноз на тестовой выборке при помощи логистической регрессии с коэффициентов регуляризации C=0.5. В ответ запишите полученный `Recall`.

LASSO регрессия для логистической регрессии юзается вот так:

`LogisticRegression(penalty='l1', solver='liblinear', random_state=your_number_random_state)`

5. При помощи кросс-валидации найдите оптимальный параметр max_depth на границах [1, 50] для дерева решений. В ответ запишите значение ROC AUC на тестовой выборке при оптимальной `максимальной глубине дерева`. Оптимизироваться нужно именно на ROC AUC.

Вам может помочь эта ссылка:
https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

6. Обучите модель бэггинга на тренировочном наборе данных для всех признаков для количества деревьев, равного 200. Чему равна `OOB ошибка` на тренировочной выборке?

7. Обучите на тренировочном датасете модель бэггинга с числом деревьев, равным 300, и сделайте прогноз на тестовой выборке. Чему равна `F1 мера`?

8. На тренировочном датасете выберите оптимальные гиперпараметры (через `Accuracy`) для дерева решений из такого множества:

 - max_depth = значения от 1 до 20 включительно
 - max_features = значения от 15 до 35 включительно
 
Чему равен `Accuracy` на тестовом наборе данных с таким набором гиперпараметров?

9. Подберите оптимальное значение  max_features = [3, 5, 10, 15, 25, 30, 35, 40] для бэггинга, где в качестве базового алгоритма будет логистическая регрессия с C=0.5. Параметр n_estimators настройте равным 25. Оптимизируемся на `Recall`. Чему равен `Precision` на тестовой выборке с лучшим гиперпараметров из тренировочной выборки?

10. Задание на бутстрап для линейной регрессии. При помощи метода бутстрапированных остатков регрессии вычислите 95% доверительный интервал для коэффициента перед переменной *Age*, где зависимая переменная *fnlwgt*. Регрессия с константой! Также вычислите 95% доверительный интервал при помощи стандартного метода в линейной регрессии при допущении о выполнении условий ГМ. В ответ запишите `отношение длины бутстрапированного доверительного интервала к длине стандартного доверительного интервала`. Число бутстрап подвыборок берем равным 10000. Фиксируем рандом своим random_state. Ответ округляем до 5 знака.